In [2]:
import folium
import requests
import json


ModuleNotFoundError: No module named 'folium'

# INSPIRE - BUILDING

In [32]:

import json
import pathlib
import requests
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", None)

In [107]:
adresse = 'Tistrupvej 14, 2650 Hvidovre'

url = f'https://api.dataforsyningen.dk/datavask/adresser?betegnelse={adresse}'
resp = requests.get(url, params={"f": "application/json"})
data = resp.json()
pretty_json = json.dumps(data, indent=4)

print(pretty_json)

{
    "kategori": "A",
    "resultater": [
        {
            "adresse": {
                "id": "74940df0-d906-4fd5-bbca-ef3392b957cf",
                "vejnavn": "Tistrupvej",
                "adresseringsvejnavn": "Tistrupvej",
                "husnr": "14",
                "supplerendebynavn": null,
                "postnr": "2650",
                "postnrnavn": "Hvidovre",
                "status": 1,
                "virkningstart": "2013-05-06T09:23:25.800Z",
                "virkningslut": null,
                "adgangsadresseid": "0a3f507c-e63f-32b8-e044-0003ba298018",
                "etage": null,
                "d\u00f8r": null,
                "href": "https://api.dataforsyningen.dk/adresser/74940df0-d906-4fd5-bbca-ef3392b957cf"
            },
            "aktueladresse": {
                "id": "74940df0-d906-4fd5-bbca-ef3392b957cf",
                "vejnavn": "Tistrupvej",
                "adresseringsvejnavn": "Tistrupvej",
                "husnr": "14",
           

In [61]:
BASE = "https://api.dataforsyningen.dk/ogcapi/features/ad_inspire"


# 1) Hent collections for at se id'er, CRS og metadata
collections_url = f"{BASE}/collections"
collections_resp = requests.get(collections_url, params={"f": "application/json"})
collections_resp.raise_for_status()
collections = collections_resp.json()


# Print til konsol hvilke collections der findes
print("Collections:")
for coll in collections.get("collections", []):
    print(" - id:", coll.get("id"), "| title:", coll.get("title"))
    # vis evt. understøttede CRS
    if "crs" in coll:
        print("   CRS:", coll["crs"])



Collections:
 - id: address | title: address
   CRS: ['#/crs']
 - id: addressareaname | title: addressareaname
   CRS: ['#/crs']
 - id: administrativeboundary | title: administrativeboundary
   CRS: ['#/crs']
 - id: adminunitname | title: adminunitname
   CRS: ['#/crs']
 - id: postaldescriptor | title: postaldescriptor
   CRS: ['#/crs']
 - id: thoroughfarename | title: thoroughfarename
   CRS: ['#/crs']


In [103]:

# 2) Definér en bbox omkring Indre København (CRS84: lon/lat)
#   (ca. 12.55±0.06, 55.67±0.04). Tilpas efter behov.
bbox = (12.46440, 55.63190,12.46449, 55.63199)


items_url = f"{BASE}/collections/address/items"
params = {
    "bbox": ",".join(map(str, bbox)),
    "limit": 100,  # paginer evt. med 'next' link i svaret
    "f": "application/geo+json",
    # Hvis din service kræver adgangstoken:
    # "token": "DIN_TOKEN_HER"
    # Hvis du vil skifte CRS (hvis serveren understøtter det), kan Part 2 syntaks være:
    # "crs": "http://www.opengis.net/def/crs/EPSG/0/4326"  # eksempel
}
resp = requests.get(items_url, params=params)
resp.raise_for_status()
data = resp.json()




In [104]:

# 3) Gem som GeoJSON
out_path = pathlib.Path("inspire_bygninger_kbh.geojson")
with out_path.open("w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False)

print(f"Skrev {out_path.resolve()} med {len(data.get('features', []))} features.")


Skrev C:\Users\misc\OneDrive - NRGi A S\NRGi Rådgivning\Kommerciel Udvikling\Klimadata\inspire_bygninger_kbh.geojson med 1 features.


In [105]:
try:
    gdf = gpd.read_file(out_path)
    gdf.head()
except Exception as e:
    print("Kunne ikke indlæse med geopandas (manglende pakke/local env). Fortsæt med GeoJSON-filen.")


In [106]:
gdf.head()

,id,ogc_fid,inspireid,alternativeidentifier,position_default,status,locator_designator_1_designator,locator_designator_1_type,locator_name_name,locator_name_type,locator_level,locator_withinscopeof_addressareaname,locator_withinscopeof_adminunitname,locator_withinscopeof_postaldescriptor,locator_withinscopeof_thoroughfarename,locator_designator_2_designator,locator_designator_2_type,locator_designator_3_designator,locator_designator_3_type,validfrom,validto,beginlifespanversion,endlifespanversion,parentaddress,component_addressareaname_1,component_addressareaname_2,component_addressareaname_3,component_adminunitname_1,component_adminunitname_2,component_adminunitname_3,component_adminunitname_4,component_adminunitname_5,component_adminunitname_6,component_postaldescriptor,component_thoroughfarename,parcel,building,position_specification,position_method,geometry
0,address.349612,349612,0a3f507c-e63f-32b8-e044-0003ba298018,None,1,current,14,entrancedooridentifier,None,None,accesslevel,None,https://api.dataforsyningen.dk/ogcapi/features...,https://api.dataforsyningen.dk/ogcapi/features...,https://api.dataforsyningen.dk/ogcapi/features...,None,flooridentifier,None,unitidentifier,2018-07-04 16:00:00+00:00,None,2018-07-04 16:00:00+00:00,None,None,None,None,None,None,https://api.dataforsyningen.dk/ogcapi/features...,https://api.dataforsyningen.dk/ogcapi/features...,None,None,None,https://api.dataforsyningen.dk/ogcapi/features...,https://api.dataforsyningen.dk/ogcapi/features...,https://api.dataforsyningen.dk/ogcapi/features...,https://api.dataforsyningen.dk/ogcapi/features...,addressarea,byotherparty,POINT (12.46443 55.63197)


# HAVVAND

In [1]:
import requests
import xml.etree.ElementTree as ET

# Funktion til at finde højden, hvor oversvømmelsen starter
def find_oversvommelseshøjde(latitude, longitude):
    bbox = f"{longitude - 0.005},{latitude - 0.005},{longitude + 0.005},{latitude + 0.005}"
    wms_url = 'https://api.dataforsyningen.dk/dhm?service=WMS&request=GetCapabilities&token=30494b8c7d48e71467a3bca51afaf457'

    styles = [
        'havvandpaaland_6', 'havvandpaaland_5_9', 'havvandpaaland_5_8', 'havvandpaaland_5_7',
        'havvandpaaland_5_6', 'havvandpaaland_5_5', 'havvandpaaland_5_4', 'havvandpaaland_5_3',
        'havvandpaaland_5_2', 'havvandpaaland_5_1', 'havvandpaaland_5', 'havvandpaaland_4_9',
        'havvandpaaland_4_8', 'havvandpaaland_4_7', 'havvandpaaland_4_6', 'havvandpaaland_4_5',
        'havvandpaaland_4_4', 'havvandpaaland_4_3', 'havvandpaaland_4_2', 'havvandpaaland_4_1',
        'havvandpaaland_4', 'havvandpaaland_3_9', 'havvandpaaland_3_8', 'havvandpaaland_3_7',
        'havvandpaaland_3_6', 'havvandpaaland_3_5', 'havvandpaaland_3_4', 'havvandpaaland_3_3',
        'havvandpaaland_3_2', 'havvandpaaland_3_1', 'havvandpaaland_3', 'havvandpaaland_2_9',
        'havvandpaaland_2_8', 'havvandpaaland_2_7', 'havvandpaaland_2_6', 'havvandpaaland_2_5',
        'havvandpaaland_2_4', 'havvandpaaland_2_3', 'havvandpaaland_2_2', 'havvandpaaland_2_1',
        'havvandpaaland_2_0'
    ]

    # Iterer gennem hver stil for at finde laveste niveau, hvor value_0 findes
    for style in styles:
        params = {
            'service': 'WMS',
            'request': 'GetFeatureInfo',
            'version': '1.1.1',
            'layers': 'dhm_havvandpaaland',
            'query_layers': 'dhm_havvandpaaland',
            'styles': style,
            'bbox': bbox,
            'width': '256',
            'height': '256',
            'x': '128',
            'y': '128',
            'srs': 'EPSG:4326',
            'format': 'image/png',
            'info_format': 'application/vnd.ogc.gml',
            'i': '50',
            'j': '50'
        }

        # Send forespørgslen til WMS-serveren
        response = requests.get(wms_url, params=params)
        if response.status_code != 200:
            continue
        
        try:
            root = ET.fromstring(response.text)
            value_0_element = root.find('.//value_0')

            if value_0_element is not None and value_0_element.text is not None:
                # Returner den fundne højde og stil (f.eks. vandstigning)
                value_0 = float(value_0_element.text)
                return value_0, style
        except ET.ParseError:
            continue

    # Hvis ingen oversvømmelseshøjde findes op til 6 meter, returnér fejl
    raise ValueError("Ingen oversvømmelseshøjde fundet op til 6 meter")

# Funktion til at beregne vanddybden ved forskellige stormflodshøjder
def beregn_vanddybde(value_0, vandstigning):
    if vandstigning < value_0:
        return 0  # Ingen oversvømmelse, vanddybden er 0
    else:
        return vandstigning - value_0


In [2]:

# Brug eksempelpunkter55.571534813751974, 12.288943150041455
latitude = 55.571534813751974
longitude = 12.288943150041455

# Find højden for oversvømmelse og beregn vanddybder
try:
    terrænhøjde, oversvommelseshøjde = find_oversvommelseshøjde(latitude, longitude)
    print(f"Terrænhøjde ved oversvømmelse: {terrænhøjde:.2f} m, oversvømmelseshøjde: {oversvommelseshøjde}")

    # Beregn vanddybder ved forskellige stormflodshøjder
    stormflodshøjder = [2.0, 4.0, 6.0]  # Eksempler på stormflodshøjder
    for højde in stormflodshøjder:
        vanddybde = beregn_vanddybde(terrænhøjde, højde)
        print(f"Vanddybde ved stormflodshøjde på {højde} m: {vanddybde:.2f} m")

except ValueError as e:
    print(e)


Terrænhøjde ved oversvømmelse: 4.09 m, oversvømmelseshøjde: havvandpaaland_6
Vanddybde ved stormflodshøjde på 2.0 m: 0.00 m
Vanddybde ved stormflodshøjde på 4.0 m: 0.00 m
Vanddybde ved stormflodshøjde på 6.0 m: 1.91 m


In [3]:
import requests
import xml.etree.ElementTree as ET
import math

def find_laveste_punkt(latitude, longitude):
    # Konvertering fra meter til grader
    lat_offset = 20 / 111320  # Omtrentlig konvertering for breddegrad
    lon_offset = 20 / (111320 * abs(math.cos(math.radians(latitude))))  # Konvertering for længdegrad afhængigt af breddegrad

    # Grid størrelse i grader (4x4 grid for at dække 20x20 meter)
    step_lat = lat_offset / 4
    step_lon = lon_offset / 4

    wms_url = 'https://api.dataforsyningen.dk/dhm?service=WMS&request=GetCapabilities&token=30494b8c7d48e71467a3bca51afaf457'

    laveste_vaerdi = None

    # Iterer gennem grid-punkterne
    for i in range(-2, 3):  # Fra -2 til 2 (5 punkter i alt)
        for j in range(-2, 3):  # Fra -2 til 2 (5 punkter i alt)
            lat = latitude + (i * step_lat)
            lon = longitude + (j * step_lon)

            # Udfør GetFeatureInfo-forespørgsel
            bbox = f"{lon - lon_offset},{lat - lat_offset},{lon + lon_offset},{lat + lat_offset}"
            params = {
                'service': 'WMS',
                'request': 'GetFeatureInfo',
                'version': '1.1.1',
                'layers': 'dhm_havvandpaaland',
                'query_layers': 'dhm_havvandpaaland',
                'styles': 'havvandpaaland_6',
                'bbox': bbox,
                'width': '256',
                'height': '256',
                'x': '128',
                'y': '128',
                'srs': 'EPSG:4326',
                'format': 'image/png',
                'info_format': 'application/vnd.ogc.gml',
                'i': '50',
                'j': '50'
            }

            # Send forespørgslen til WMS-serveren
            response = requests.get(wms_url, params=params)
            if response.status_code != 200:
                continue

            try:
                root = ET.fromstring(response.text)
                value_0_element = root.find('.//value_0')

                if value_0_element is not None and value_0_element.text is not None:
                    value_0 = float(value_0_element.text)
                    # Kontroller om value_0 er en gyldig værdi
                    if value_0 > -1e10 and value_0 < 1e10:  # Filtrer ekstreme værdier fra
                        if laveste_vaerdi is None or value_0 < laveste_vaerdi:
                            laveste_vaerdi = value_0
            except ET.ParseError:
                continue

    if laveste_vaerdi is None:
        raise ValueError("Ingen gyldig værdi fundet inden for 20 meter")

    return laveste_vaerdi

# Brug eksempelpunkter  55.57153793906798, 12.288919196041304
latitude = 55.57153793906798
longitude = 12.288919196041304

try:
    laveste_punkt = find_laveste_punkt(latitude, longitude)
    print(f"Den laveste værdi inden for 20 meter er: {laveste_punkt:.2f} meter")
except ValueError as e:
    print(e)


Den laveste værdi inden for 20 meter er: 4.05 meter


# EKSTREMREGN

In [4]:


def Ekstremregn_punkt(latitude, longitude):
    bbox = f"{longitude - 0.005},{latitude - 0.005},{longitude + 0.005},{latitude + 0.005}"
    wms_url = 'https://api.dataforsyningen.dk/dhm?service=WMS&request=GetCapabilities&token=30494b8c7d48e71467a3bca51afaf457'

    # Iterer gennem hver stil for at finde laveste niveau, hvor value_0 findes

    params = {
        'service': 'WMS',
        'request': 'GetFeatureInfo',
        'version': '1.1.1',
        'layers': 'dhm_bluespot_ekstremregn',
        'query_layers': 'dhm_bluespot_ekstremregn',
        'styles': 'bluespot_ekstremregn_0_090',
        'bbox': bbox,
        'width': '256',
        'height': '256',
        'x': '128',
        'y': '128',
        'srs': 'EPSG:4326',
        'format': 'image/png',
        'info_format': 'application/vnd.ogc.gml',
        'i': '50',
        'j': '50'
    }

    # Send forespørgslen til WMS-serveren
    response = requests.get(wms_url, params=params)

    # Kontroller om forespørgslen lykkedes
    if response.status_code != 200:
        print(f"Fejl ved forespørgsel: {response.status_code}")
        raise ValueError("Forespørgsel til WMS-server mislykkedes")

    # Udskriv og parse XML-responsen
    try:
        root = ET.fromstring(response.text)
        value_0_element = root.find('.//value_0')

        if value_0_element is not None and value_0_element.text is not None:
            # Returner den fundne værdi
            value_0 = float(value_0_element.text)
            print(f"Value 0: {value_0}")
            return (value_0)
        else:
            print("Ingen gyldig value_0 fundet")
    except ET.ParseError:
        print('Kunne ikke parse XML responsen korrekt')



In [5]:
# Brug eksempelpunkter  55.70716827070835, 12.509348376132367  55.70639603118447, 12.507620912023917
latitude = 55.706399843591505
longitude = 12.507406643321174

Ekstremregn_punkt(latitude, longitude)

# Brug eksempelpunkter  55.70716827070835, 12.509348376132367  55.70639603118447, 12.507620912023917  55.70651040323336, 12.50813966783056
latitude = 55.70651040323336
longitude = 12.50813966783056

Ekstremregn_punkt(latitude, longitude)

# Brug eksempelpunkter  55.70716827070835, 12.509348376132367  55.70639603118447, 12.507620912023917  55.70710767404985, 12.509533542128409
latitude = 55.70710767404985
longitude = 12.509533542128409

Ekstremregn_punkt(latitude, longitude)

# Brug eksempelpunkter  55.70716827070835, 12.509348376132367  55.70639603118447, 12.507620912023917  55.70730151664934, 12.510275844218578
latitude = 55.70730151664934
longitude = 12.510275844218578

Ekstremregn_punkt(latitude, longitude)

Value 0: 0.061078947
Value 0: 0.038934451
Value 0: 0.026233928
Value 0: 0.08646445


0.08646445

In [6]:
import requests
import xml.etree.ElementTree as ET
import math

def find_laveste_punkt(latitude, longitude):
    # Konvertering fra meter til grader
    lat_offset = 20 / 111320  # Omtrentlig konvertering for breddegrad
    lon_offset = 20 / (111320 * abs(math.cos(math.radians(latitude))))  # Konvertering for længdegrad afhængigt af breddegrad

    # Grid størrelse i grader (4x4 grid for at dække 20x20 meter)
    step_lat = lat_offset / 4
    step_lon = lon_offset / 4

    wms_url = 'https://api.dataforsyningen.dk/dhm?service=WMS&request=GetCapabilities&token=30494b8c7d48e71467a3bca51afaf457'

    laveste_vaerdi = None

    # Iterer gennem grid-punkterne
    for i in range(-2, 3):  # Fra -2 til 2 (5 punkter i alt)
        for j in range(-2, 3):  # Fra -2 til 2 (5 punkter i alt)
            lat = latitude + (i * step_lat)
            lon = longitude + (j * step_lon)

            # Udfør GetFeatureInfo-forespørgsel
            bbox = f"{lon - lon_offset},{lat - lat_offset},{lon + lon_offset},{lat + lat_offset}"
            params = {
                'service': 'WMS',
                'request': 'GetFeatureInfo',
                'version': '1.1.1',
                'layers': 'dhm_havvandpaaland',
                'query_layers': 'dhm_havvandpaaland',
                'styles': 'havvandpaaland_6',
                'bbox': bbox,
                'width': '256',
                'height': '256',
                'x': '128',
                'y': '128',
                'srs': 'EPSG:4326',
                'format': 'image/png',
                'info_format': 'application/vnd.ogc.gml',
                'i': '50',
                'j': '50'
            }

            # Send forespørgslen til WMS-serveren
            response = requests.get(wms_url, params=params)
            if response.status_code != 200:
                continue

            try:
                root = ET.fromstring(response.text)
                value_0_element = root.find('.//value_0')

                if value_0_element is not None and value_0_element.text is not None:
                    value_0 = float(value_0_element.text)
                    # Kontroller om value_0 er en gyldig værdi
                    if value_0 > -1e10 and value_0 < 1e10:  # Filtrer ekstreme værdier fra
                        if laveste_vaerdi is None or value_0 < laveste_vaerdi:
                            laveste_vaerdi = value_0
            except ET.ParseError:
                continue

    if laveste_vaerdi is None:
        raise ValueError("Ingen gyldig værdi fundet inden for 20 meter")

    return laveste_vaerdi

# Brug eksempelpunkter  55.57153793906798, 12.288919196041304
latitude = 55.57153793906798
longitude = 12.288919196041304

try:
    laveste_punkt = find_laveste_punkt(latitude, longitude)
    print(f"Den laveste værdi inden for 20 meter er: {laveste_punkt:.2f} meter")
except ValueError as e:
    print(e)


Den laveste værdi inden for 20 meter er: 4.05 meter


In [7]:
import requests
import xml.etree.ElementTree as ET

# Funktion til at finde højden, hvor oversvømmelsen starter
def find_oversvommelseshøjde(latitude, longitude):
    bbox = f"{longitude - 0.005},{latitude - 0.005},{longitude + 0.005},{latitude + 0.005}"
    wms_url = 'https://api.dataforsyningen.dk/dhm?service=WMS&request=GetCapabilities&token=30494b8c7d48e71467a3bca51afaf457'

    # Iterer gennem hver stil for at finde laveste niveau, hvor value_0 findes
   
    params = {
        'service': 'WMS',
        'request': 'GetFeatureInfo',
        'version': '1.1.1',
        'layers': 'dhm_bluespot_ekstremregn',
        'styles': 'bluespot_ekstremregn_0_150',
        'bbox': bbox,
        'width': '256',
        'height': '256',
        'x': '128',
        'y': '128',
        'srs': 'EPSG:4326',
        'format': 'image/png',
        'info_format': 'application/vnd.ogc.gml',
        'i': '50',
        'j': '50'
    }

    # Send forespørgslen til WMS-serveren
    response = requests.get(wms_url, params=params)
    if response.status_code != 200:
        print('No')
    
    try:
        root = ET.fromstring(response.text)
        value_0_element = root.find('.//value_0')

        if value_0_element is not None and value_0_element.text is not None:
            # Returner den fundne højde og stil (f.eks. vandstigning)
            value_0 = float(value_0_element.text)
            return value_0, style
    except ET.ParseError:
        print('No No')

    # Hvis ingen oversvømmelseshøjde findes op til 6 meter, returnér fejl
    raise ValueError("Ingen oversvømmelseshøjde fundet op til 6 meter")



In [4]:

# Brug eksempelpunkter55.571534813751974, 12.288943150041455
latitude = 55.571534813751974
longitude = 12.288943150041455

# Find højden for oversvømmelse og beregn vanddybder
try:
    terrænhøjde, oversvommelseshøjde = find_oversvommelseshøjde(latitude, longitude)
    print(f"Terrænhøjde ved oversvømmelse: {terrænhøjde:.2f} m, oversvømmelseshøjde: {oversvommelseshøjde}")

    # Beregn vanddybder ved forskellige stormflodshøjder
    stormflodshøjder = [2.0, 4.0, 6.0]  # Eksempler på stormflodshøjder
    for højde in stormflodshøjder:
        vanddybde = beregn_vanddybde(terrænhøjde, højde)
        print(f"Vanddybde ved stormflodshøjde på {højde} m: {vanddybde:.2f} m")

except ValueError as e:
    print(e)


Ingen oversvømmelseshøjde fundet op til 6 meter


In [45]:
import requests
import xml.etree.ElementTree as ET
import math

def find_laveste_punkt(latitude, longitude):
    # Konvertering fra meter til grader
    lat_offset = 20 / 111320  # Omtrentlig konvertering for breddegrad
    lon_offset = 20 / (111320 * abs(math.cos(math.radians(latitude))))  # Konvertering for længdegrad afhængigt af breddegrad

    # Grid størrelse i grader (4x4 grid for at dække 20x20 meter)
    step_lat = lat_offset / 4
    step_lon = lon_offset / 4

    wms_url = 'https://api.dataforsyningen.dk/dhm?service=WMS&request=GetCapabilities&token=30494b8c7d48e71467a3bca51afaf457'

    laveste_vaerdi = None

    # Iterer gennem grid-punkterne
    for i in range(-2, 3):  # Fra -2 til 2 (5 punkter i alt)
        for j in range(-2, 3):  # Fra -2 til 2 (5 punkter i alt)
            lat = latitude + (i * step_lat)
            lon = longitude + (j * step_lon)

            # Udfør GetFeatureInfo-forespørgsel
            bbox = f"{lon - lon_offset},{lat - lat_offset},{lon + lon_offset},{lat + lat_offset}"
            params = {
                'service': 'WMS',
                'request': 'GetFeatureInfo',
                'version': '1.1.1',
                'layers': 'dhm_bluespot_ekstremregn',
                'query_layers': 'dhm_bluespot_ekstremregn',
                'styles': 'bluespot_ekstremregn_0_150',
                'bbox': bbox,
                'width': '256',
                'height': '256',
                'x': '128',
                'y': '128',
                'srs': 'EPSG:4326',
                'format': 'image/png',
                'info_format': 'application/vnd.ogc.gml',
                'i': '50',
                'j': '50'
            }

            # Send forespørgslen til WMS-serveren
            response = requests.get(wms_url, params=params)
            if response.status_code != 200:
                continue

            try:
                root = ET.fromstring(response.text)
                value_0_element = root.find('.//value_0')

                if value_0_element is not None and value_0_element.text is not None:
                    value_0 = float(value_0_element.text)
                    # Kontroller om value_0 er en gyldig værdi
                    if value_0 > -1e10 and value_0 < 1e10:  # Filtrer ekstreme værdier fra
                        if laveste_vaerdi is None or value_0 < laveste_vaerdi:
                            laveste_vaerdi = value_0
            except ET.ParseError:
                continue

    if laveste_vaerdi is None:
        raise ValueError("Ingen gyldig værdi fundet inden for 20 meter")

    return laveste_vaerdi

# Brug eksempelpunkter  55.57153793906798, 12.288919196041304
latitude = 55.57153793906798
longitude = 12.288919196041304

try:
    laveste_punkt = find_laveste_punkt(latitude, longitude)
    print(f"Den laveste værdi inden for 20 meter er: {laveste_punkt:.2f} meter")
except ValueError as e:
    print(e)


Den laveste værdi inden for 20 meter er: 0.00 meter
